In [1]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
from hups_utilities import *
import random
import numpy as np
from deepNeuralNetwork import *
from collections import OrderedDict
from collections import Counter
from dnn_app_utils_v2 import *
from game import *

In [14]:
gto_players = []
neural_players = []
games = []
number_games = 1000
number_hands = 10000
neural_chips = 0
neural_chip_results = []

for i in range(number_games):
    gto_player = Player(True, 2)
    neural_player = Player(True, 3)
    game = Game([neural_player, gto_player])
    game.output = False
    
    gto_player.game, neural_player.game = game, game
    
    gto_players.append(gto_player)
    neural_players.append(neural_player)
    games.append(game)
    
for h in range(number_hands):
    neural_chips = 0
    for g in range(number_games):
        games[g].nextHand()
        neural_chips += neural_players[g].chips
    neural_chip_results.append(neural_chips/number_games)
    if h % 10 == 0:
        print neural_chip_results[h]

0
5
8


KeyboardInterrupt: 

In [13]:
player1 = Player(True, 2)
player2 = Player(True, 3)
game = Game([player1, player2])
player1.game = game
player2.game = game
game.output = False
hands = 0

for i in range(hands):
    game.nextHand()
    if i % 100000 == 0:
        print player2.chips
    
'''
print player1.preflopHandOdds
print player2.preflopHandOdds

print createPreflopOrderedRange(game.getHandPercentages())
print (createPreflopOrderedRange(np.loadtxt("data", unpack=True)))
'''

'\nprint player1.preflopHandOdds\nprint player2.preflopHandOdds\n\nprint createPreflopOrderedRange(game.getHandPercentages())\nprint (createPreflopOrderedRange(np.loadtxt("data", unpack=True)))\n'

In [12]:
class Player:
    game = None
    isAI = True
    AIlevel = 1
    chips = 0
    bankroll = 999000
    hands = 0
    neuralParameters = None
    raisePercent = 1./3
    callPercent = 2./3
    hand = []
    preflopHandOdds = []
    currentRange = []
    decisions = ['Fold', 'Call', 'Check', 'Bet', 'Raise']
    aiType = {1 : 'Calling Station', 2 : 'GTO', 3 : 'NeuralGTO'}
    
    def __init__(self, isAI = True, AIlevel = 1):
        self.isAI = isAI
        self.AIlevel = AIlevel
        if isAI:
            #self.preflopHandOdds = OrderedDict(createPreflopOrderedRange(np.loadtxt("data", unpack=True)))
        
            '''
            if AIlevel in [2, 3]:
                self.updateHandRange()
            '''
            
    def newHand(self):
        if self.hands % 10 == 0:
            self.updateHandRange()
        self.currentRange = self.preflopHandOdds
        self.hands += 1
    
    def generateHandAbbreviation(self):
        suited = 'o'
        if self.hand[0].suit == self.hand[1].suit:
            suited = 's'
        return self.hand[0].rank + self.hand[1].rank + suited
    
    def decideLevel1(self, previousAction, raiseAmount = 0):
        if previousAction == None:
            if raiseAmount > 0: return 'Call'
            else: return 'Check'
        elif previousAction == 'Bet' or previousAction == 'Raise': return 'Call'
        elif previousAction == 'Call' or previousAction == 'Check': return 'Check'
        
            
    def decideLevel2(self, previousAction, raiseAmount = 0):
        n = len(self.currentRange)
        odds = self.currentRange[self.generateHandAbbreviation()]
        
        if previousAction in ['Call', 'Check', None]:
            if odds > self.currentRange.values()[int(round(self.raisePercent * n))]: 
                self.currentRange = OrderedDict(self.currentRange.items()[:int(round(self.raisePercent * n))])
                return 'Bet'
            elif raiseAmount > 0: return 'Call' 
            else: return 'Check'
        elif previousAction == 'Bet' or previousAction == 'Raise':
            if odds > self.currentRange.values()[int(round(self.raisePercent * self.callPercent * n))]: 
                self.currentRange = OrderedDict(self.currentRange.items()[:int(round(self.raisePercent * self.callPercent * n))])
                return 'Raise'
            elif odds > self.currentRange.values()[int(round(self.callPercent * n))]:
                self.currentRange = OrderedDict(self.currentRange.items()[:int(round(self.callPercent * n))])
                return 'Call'
            else:
                return 'Fold'
        assert False
    
    def updateHandRange(self):
        
        #The AI that doesn't use a neural network would have a huge advantage if we didn't randomize their starting range
        if self.AIlevel == 2:
            self.preflopHandOdds = createPreflopOrderedRange(self.game.getHandPercentages())
        if self.AIlevel == 3:
            currentHandPercentages = np.array(self.game.getHandPercentages()).reshape(1,169)
            overallHandPercentages = np.loadtxt("data", unpack=True).reshape(1, 169)

            train_x = currentHandPercentages
            train_y = overallHandPercentages

            n_x = train_x.shape[0]
            n_y = train_y.shape[0]
            layers_dims = [n_x, 50, 12, 5, n_y] #  This determines how many hidden layers the network will use

            parameters = L_layer_model(train_x, train_y, layers_dims, learning_rate = .2, num_iterations = 100, print_cost = False, plotGraph = False, params = self.neuralParameters)
            self.neuralParameters = parameters
            
            AL, caches = L_model_forward(currentHandPercentages, parameters)
            
            self.preflopHandOdds = OrderedDict(createPreflopOrderedRange(AL.reshape(13, 13)))

    def decide(self, previousAction, raiseAmount = 0):
        if self.isAI:
            if self.AIlevel == 1: return self.decideLevel1(previousAction, raiseAmount)
            elif self.AIlevel in [2,3]: return self.decideLevel2(previousAction, raiseAmount)
            

Player1 = Player(True, 2)
Player2 = Player(True, 3)
game = Game([Player1,Player2])
Player1.game = game
Player2.game = game
game.output = False
hands = 1
neuralCosts = []
currentCosts = []
iterations = []
for x in range(hands):
    game.nextHand()
    if game.outputDict['Betting'].count('Raise') > 4 and game.outputDict['Betting'].count('Call') > 0:
        print game.outputDict['Text']
        break



In [ ]:
def printStartingHandPercentages(number):
    hands = number
    game.output = False
    for x in range(hands):
        game.nextHand()
    # This is for saving the data, but since running 10,000,000 hands took a few hours, I'm happy to stick with the results I have
    '''
    f = open("data", "w")
    np.savetxt(f, np.array([float(x) for x in np.array(startingHands).reshape(169,)]).reshape(13,13), fmt="%.3f", header="starting hands")
    f.close()
    hands = np.loadtxt("data", unpack=True)
    '''
    print "After " + str(game.handsPlayed) + " hands..."
    #print ['{:02.4f}'.format(game.getHandTypeResults()[x] * 100) for x in range(9)]

    print "      " + str(["  " + deck.rank[y] + "  " for y in range(13)])
    for y in range(13):
        #print ['{:04.2f}'.format(game.preflopHandsPlayed[y][x] * (1326 / 2)  / game.handsPlayed).zfill(5) for x in range(13)]
        print str([deck.rank[y]]) + " " + str(['{:03.3f}'.format(game.getHandPercentages()[y][x]) for x in range(13)])

In [ ]:
np.loadtxt("data", unpack=True).reshape(1, 169)

In [ ]:
testPokerStartingHands()